In [1]:
import skimage.io
import dask.array as da
import dask
from pathlib import Path
import napari
from natsort import natsorted
import numpy as np
from napari_flim_phasor_calculator._reader import get_current_tz


Define read 2d and 3d functions

In [2]:

folder_path = r"C:\Users\mazo260d\Desktop\Conni_BiA_PoL\embryo_FLIM_data\raw_data_embryo_stack_3tps_43pl_2ch\output_as_tif"

folder_path = Path(folder_path)

def read_tif_data_2D(path):
    from skimage.io import imread
    image = imread(path)
    return image

def read_tif_data_3D(file_paths, image_slice_max_shape):
    file_paths = natsorted(file_paths)
    z_list = []
    for file_path in file_paths:
        if file_path.suffix == '.tif':
            current_t, current_z = get_current_tz(file_path)
            data = read_tif_data_2D(file_path)
            if current_z is not None:
                image = np.zeros(image_slice_max_shape, dtype=data.dtype)
                image[:data.shape[0], :data.shape[1], :data.shape[2], :data.shape[3]] = data
                z_list.append(image)
    image_3D = np.stack(z_list)
    # move channel and microtime to the beginning (putting z behind them): from (z, ch, mt, y, x) to (ch, mt, z, y, x)
    image_3D = np.moveaxis(image_3D, [-4, -3], [0, 1])
    return image_3D

Get max slice shape

In [3]:
# Read all slices to get slice max shape and dtype (not ideal but for now it is OK)
slice_shape_list = []
for file_path in folder_path.iterdir():
    if file_path.suffix == '.tif':
        image_2D = read_tif_data_2D(file_path)
        slice_shape_list.append(image_2D.shape)
slice_max_shape = max(slice_shape_list)
print('last_slice_shape = ', image_2D.shape,'image_dtype = ', image_2D.dtype)
print('max_slice_shape = ', slice_max_shape)

last_slice_shape =  (2, 132, 256, 256) image_dtype =  uint16
max_slice_shape =  (2, 138, 256, 256)


Get max z slices and max t timepoints from filepaths

In [4]:
file_paths = [file_path for file_path in folder_path.iterdir() if file_path.suffix == '.tif']

# Get max z slices by reading all file names
def get_max_zslices(file_paths):
    max_z = max([get_current_tz(file_path) for file_path in file_paths if file_path.suffix == '.tif'])[1]
    if max_z is None:
        return 1
    return max_z
def get_max_time(file_paths):
    max_time = max([get_current_tz(file_path) for file_path in file_paths if file_path.suffix == '.tif'])[0]
    if max_time is None:
        return 1
    return max_time

max_z = get_max_zslices(file_paths)
print('max_z slices = ', max_z)
max_t = get_max_time(file_paths)
print('max_time = ', max_t)

max_z slices =  43
max_time =  3


Get stack max shape

In [5]:
image_stack_shape = (*slice_max_shape[:-2], max_z, *slice_max_shape[-2:])
image_stack_shape


(2, 138, 43, 256, 256)

Make dask array

In [10]:
t_path_list = []
z_path_list = []
file_paths = natsorted(file_paths)
previous_t = 1
for file_path in file_paths:
    if file_path.suffix == '.tif':
        current_t, current_z = get_current_tz(file_path)
        print(current_t, current_z)
        if current_t is not None:
            if current_t > previous_t:
                t_path_list.append(z_path_list)
                z_path_list = []
                previous_t = current_t
            z_path_list.append(file_path)

# If no timepoints, z+path_list is file_paths
if current_t is None:
    z_path_list = file_paths

# Append last timepoint
t_path_list.append(z_path_list)


1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
1 42
1 43
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
2 28
2 29
2 30
2 31
2 32
2 33
2 34
2 35
2 36
2 37
2 38
2 39
2 40
2 41
2 42
2 43
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
3 25
3 26
3 27
3 28
3 29
3 30
3 31
3 32
3 33
3 34
3 35
3 36
3 37
3 38
3 39
3 40
3 41
3 42
3 43


In [11]:
z_path_list

[WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/embryo_FLIM_data/raw_data_embryo_stack_3tps_43pl_2ch/output_as_tif/embryo_43pl_2ch_3tps_t3_z1.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/embryo_FLIM_data/raw_data_embryo_stack_3tps_43pl_2ch/output_as_tif/embryo_43pl_2ch_3tps_t3_z2.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/embryo_FLIM_data/raw_data_embryo_stack_3tps_43pl_2ch/output_as_tif/embryo_43pl_2ch_3tps_t3_z3.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/embryo_FLIM_data/raw_data_embryo_stack_3tps_43pl_2ch/output_as_tif/embryo_43pl_2ch_3tps_t3_z4.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/embryo_FLIM_data/raw_data_embryo_stack_3tps_43pl_2ch/output_as_tif/embryo_43pl_2ch_3tps_t3_z5.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/embryo_FLIM_data/raw_data_embryo_stack_3tps_43pl_2ch/output_as_tif/embryo_43pl_2ch_3tps_t3_z6.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/embryo_FLIM_data

In [12]:
for z_path_list in t_path_list:
    print(z_path_list)

[WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/embryo_FLIM_data/raw_data_embryo_stack_3tps_43pl_2ch/output_as_tif/embryo_43pl_2ch_3tps_t1_z1.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/embryo_FLIM_data/raw_data_embryo_stack_3tps_43pl_2ch/output_as_tif/embryo_43pl_2ch_3tps_t1_z2.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/embryo_FLIM_data/raw_data_embryo_stack_3tps_43pl_2ch/output_as_tif/embryo_43pl_2ch_3tps_t1_z3.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/embryo_FLIM_data/raw_data_embryo_stack_3tps_43pl_2ch/output_as_tif/embryo_43pl_2ch_3tps_t1_z4.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/embryo_FLIM_data/raw_data_embryo_stack_3tps_43pl_2ch/output_as_tif/embryo_43pl_2ch_3tps_t1_z5.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/embryo_FLIM_data/raw_data_embryo_stack_3tps_43pl_2ch/output_as_tif/embryo_43pl_2ch_3tps_t1_z6.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/embryo_FLIM_data/raw_d

In [13]:
read_tif_data_3D_delayed = dask.delayed(read_tif_data_3D, pure=True)

lazy_images = [read_tif_data_3D_delayed(z_path_list, image_slice_max_shape=slice_max_shape) for z_path_list in t_path_list]

arrays = [da.from_delayed(lazy_image,           # Construct a small Dask array
                          dtype=image_2D.dtype,   # for every lazy value
                          shape=image_stack_shape)
          for lazy_image in lazy_images]

stack = da.stack(arrays, axis=0)                # Stack all small Dask arrays into one
print('stack shape and chunks = ', stack.shape, '\n', stack.chunks)

stack shape and chunks =  (3, 2, 138, 43, 256, 256) 
 ((1, 1, 1), (2,), (138,), (43,), (256,), (256,))


In [52]:
stack.chunks

((1, 1, 1), (2,), (138,), (43,), (256,), (256,))

In [97]:
chunk_size_MBytes = np.cumprod(stack.chunksize)[-1] * stack.itemsize / 1e6 #MB per chunk
chunk_size_MBytes

1555.562496

In [98]:
if chunk_size_MBytes > 1e3: # if larger than 1GB (1000MB)
    stack = stack.rechunk({-3: 'auto'}, block_size_limit=1e9)

In [99]:
viewer = napari.Viewer()

In [100]:
viewer.add_image(stack, channel_axis=1)

[<Image layer 'Image' at 0x17a4dcb8520>,
 <Image layer 'Image [1]' at 0x17a3da0be50>]